In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

path = "refFlat.txt"
path_fa = "result.fa"

In [2]:
def read_fasta(file_path):
    sequences = {}
    seq_id = None
    current_seq = ""

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):  
                if seq_id is not None:
                    sequences[seq_id] = current_seq
                    current_seq = ""
                seq_id = line[1:]  
            else:
                current_seq += line

        if seq_id is not None and current_seq != "":
            sequences[seq_id] = current_seq

    return sequences



In [3]:
import re

def split_by_space(text):
    return re.split('\s+', text)

In [4]:
def gene_data(text_list):
    contents =[]
    for i in range(len(text_list)-1):
        contents.append(text_list[i])
    return contents

In [5]:
def result(path):
    with open(path, "r", encoding="utf-8") as file:
        lines =file.readlines()
    result = []
    for line in lines:
        line =split_by_space(line)
        contents=gene_data(line)
        result.append(contents)
    return result
    

In [6]:
gene_data =result(path)
col_names =['geneName','name','chrom','strand','txStart','txEnd','cdsStart','cdsEnd','exonCount','exonStarts','exonEnds']
gene_df=pd.DataFrame(gene_data,columns=col_names)

In [7]:
base_data = read_fasta(path_fa)

In [8]:
df=gene_df[gene_df.duplicated('name', keep=False)]

In [9]:
def target_base_CT_plus(seq):
    match = re.findall('(?=(\w{20}GG))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][0:3]
        if target =='CAA'or target =='CAG'or target =='CGA':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [10]:
def target_base_AG_plsu(seq):
    match = re.findall('(?=(CC\w{20}))', seq)
    target_list=[]
    print(match)
    for i in range(len(match)):
        target = match[i][-3:]
        print(target)
        if target =='TGG':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [11]:
def target_start(target_list,qua):    
    result = []
    for i in target_list:
        matches = re.finditer(i, qua)
        for match in matches:
            start_index = match.start()
            result.append(start_index)
    return result

In [12]:
def target_gene_start(name,target):
    data = gene_df[gene_df['name']==name]
    num = data['txStart'].astype(int).to_numpy()
    target = np.array(target)
    result = target+num
    #ここ-1が必要かもしれない？
    return result
    

In [13]:
samp = base_data["NM_001385633::chr7:44239875-44254175"]

In [14]:
target_gene_start("NM_001385633",target_start(target_base_CT_plus(samp),samp))

array([44240815, 44240885, 44241000, 44241367, 44241812, 44242688,
       44242785, 44243474, 44243482, 44243814, 44244057, 44244134,
       44244358, 44244421, 44244686, 44245681, 44245743, 44245822,
       44246523, 44246563, 44246622, 44246632, 44246729, 44246920,
       44246925, 44246957, 44247801, 44247903, 44248299, 44248492,
       44248645, 44249281, 44249589, 44250214, 44251317, 44251431,
       44251549, 44252026, 44252959, 44253235, 44253896, 44253952,
       44253991, 44253995])

In [15]:
def exon_data(name,sequence_data):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    exon_seq_list = []
    start_list = [num for num in start.split(",")]
    end_list = [num for num in end.split(",")]
    for s in range(len(start_list)-1):
        start_num = int(start_list[s])- set_num
        
        end_num= int(end_list[s]) - set_num
    
        
        exon_seq =sequence_data[start_num:end_num]
        exon_seq_list.append(exon_seq)
        
    
    
    return ''.join(exon_seq_list)

In [16]:
def where_start_codon(name,exon_data):
    data = gene_df[gene_df['name']==name]
    cdsStart =int(data['cdsStart'].to_list()[0])
    set_num =int(data['cdsStart'].to_list()[0])-int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    for s in range(len(start)-1):
        if (int(start[s]) <= cdsStart<=int(end[s])):
            cds =s
            
    return exon_num


In [17]:
def start_codon(name,exon_data,exon_num):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])-int(data['txStart'].to_list()[0])
    CDs =''
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    cds_list=[]
    if exon_num == 0:
            cds_list.append(exon_data[set_num:int(end[0])-int(data['txStart'].to_list()[0])])
            for s in range(1,len(start)-1):
                cds_list.append(exon_data[int(start[s])-int(data['txStart'].to_list()[0]):int(end[s])-int(data['txStart'].to_list()[0])])
              
    
    #elseは要修正
    else:     
        start_num = int(data['cdsStart'].to_list()[0])- int(start[exon_num])

        for s in range(exon_num):
            start_num += int(end[s])-int(start[s])
        cds_list.append(exon_data[start_num:end[exson_num]])
        
        for t in range(exon_num+1,len(start)-1):
                cds_list.append(exon_data[start[t]:end[t]])
        
        
    return ''.join(cds_list)



In [18]:
def end_codon(cds_seq):
    matches = re.finditer('(?=(CAA)|(?=(CAG))|(?=(CGA))|(?=(TGG)))', cds_seq)
    result=[]
    for match in matches:
        start_pos = match.start() 
        
        if (start_pos % 3) == 0:
            result.append(start_pos)
    return result

In [22]:
def add_num(name,exon_num):#if分以降が使えそう
    
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    if exon_num == 0:
        add_result= set_num
    else:
        add_result= int(start[exon_num])
        
    return add_result

In [23]:
def add_num_list(name,exon_data):#修正が必要
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_list=[]
    num = 0
    for s in range(len(start)-1):
        element_list=[]
        element_list.append(num)
        
        num += int(end[s])-int(start[s])
        
        element_list.append(num)
        add_list.append(element_list)
        
    return add_list

In [ ]:
start_codon('NM_001385633', base_data["NM_001385633::chr7:44239875-44254175"],0)

In [31]:
end_num =end_codon(start_codon('NM_001385633', base_data["NM_001385633::chr7:44239875-44254175"],0))

In [24]:
add_num_list('NM_001385633', base_data["NM_001385633::chr7:44239875-44254175"])

[[0, 1307], [1307, 2415], [2415, 2610], [2610, 2670], [2670, 5553]]

In [55]:
def add_num(name,number_list, interval_list):
    indices = []
    for number in number_list:
        for index, interval in enumerate(interval_list):
            if int(interval[0]) <= int(number) < int(interval[1]):
                indices.append(index)
                break
            
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_result=[]
    for i in indices:
        if i == 0:
            add_result.append(set_num)
        else:
            add_result.append(int(start[i]))
        
    return add_result

In [57]:
ad_lis=add_num('NM_001385633',end_num, add_num_list('NM_001385633', base_data["NM_001385633::chr7:44239875-44254175"]))

In [58]:
end_num = np.array(end_num)

In [60]:
end_num

array([  21,   39,  138,  525,  561,  576,  744,  804,  807,  816,  819,
        831,  975, 1074, 1485, 1488, 1542, 1659, 1704, 1950, 2049, 2118,
       2169, 2427, 2439, 2484, 2493, 2499, 2502, 2559, 2655, 2721, 2733,
       2736, 2760, 2826, 2841, 2970, 3000, 3054, 3060, 3093, 3234, 3246,
       3363, 3408, 3423, 3444, 3465, 3519, 3525, 3543, 3567, 3603, 3612,
       3648, 3708, 3855, 3903, 3915, 3921, 4005, 4014, 4179, 4251, 4275,
       4284, 4296, 4380, 4581, 4695, 4698, 4797, 4821, 4836, 4878, 4896,
       4902, 4911, 4935, 5013])

In [62]:
ad_lis = np.array(ad_lis)

In [63]:
ad_lis

array([44240309, 44240309, 44240309, 44240309, 44240309, 44240309,
       44240309, 44240309, 44240309, 44240309, 44240309, 44240309,
       44240309, 44240309, 44244584, 44244584, 44244584, 44244584,
       44244584, 44244584, 44244584, 44244584, 44244584, 44247804,
       44247804, 44247804, 44247804, 44247804, 44247804, 44247804,
       44250292, 44251292, 44251292, 44251292, 44251292, 44251292,
       44251292, 44251292, 44251292, 44251292, 44251292, 44251292,
       44251292, 44251292, 44251292, 44251292, 44251292, 44251292,
       44251292, 44251292, 44251292, 44251292, 44251292, 44251292,
       44251292, 44251292, 44251292, 44251292, 44251292, 44251292,
       44251292, 44251292, 44251292, 44251292, 44251292, 44251292,
       44251292, 44251292, 44251292, 44251292, 44251292, 44251292,
       44251292, 44251292, 44251292, 44251292, 44251292, 44251292,
       44251292, 44251292, 44251292])

In [64]:
a = end_num+ad_lis

In [65]:
b=target_gene_start("NM_001385633",target_start(target_base_CT_plus(samp),samp))

In [66]:
target_gene_start("NM_001385633",target_start(target_base_CT_plus(samp),samp))

array([44240815, 44240885, 44241000, 44241367, 44241812, 44242688,
       44242785, 44243474, 44243482, 44243814, 44244057, 44244134,
       44244358, 44244421, 44244686, 44245681, 44245743, 44245822,
       44246523, 44246563, 44246622, 44246632, 44246729, 44246920,
       44246925, 44246957, 44247801, 44247903, 44248299, 44248492,
       44248645, 44249281, 44249589, 44250214, 44251317, 44251431,
       44251549, 44252026, 44252959, 44253235, 44253896, 44253952,
       44253991, 44253995])

In [67]:
common_elements = np.intersect1d(a,b)

In [68]:
common_elements 

array([44240885])